In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from datetime import datetime

import googletrans

import requests
from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import numpy as np


import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

from wordcloud import WordCloud
from langdetect import detect
from textblob import TextBlob

In [2]:
def url_request(url):
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome()
    driver.get(url)    
    return driver

In [3]:
driver = url_request('https://www.rt.com')

In [4]:
def key_word(word):
    
    cookies= driver.find_element(By.PARTIAL_LINK_TEXT,'Accept cookies').click()
    
    
    search_bar = driver.find_element(By.CLASS_NAME, "js-search-input")
    search_bar.send_keys(word)
    search_bar.send_keys(Keys.RETURN)

    lst_link=[]
    try:

        while len(lst_link)< 20:
            lnks       = driver.find_elements(By.TAG_NAME,"a")
            lst_rt     = [lnk.get_attribute('href') for lnk in lnks]
            lst_rt_1   = [url for url in lst_rt if 'https://www.rt.com/news/5' in str(url)]
            lst_link   = [i for i in lst_rt_1]
            
            try:
                more_button= driver.find_element(By.ID,'listingBtn')
                driver.execute_script("window.scrollTo(0,document.body.scrollHeight)",more_button)
                more_button.click()
            except:
                break
                
            lst_link=list(set(lst_link))

            list_soup = []
            list_title=[]
            for url in lst_link:
                html  = requests.get(url)
                soup  = BeautifulSoup(html.content, "html.parser")
                title_element = soup.find('title')
                title = title_element.text
                list_soup.append(soup)
                list_title.append(title)
                
            art_list = []  
            for soup in list_soup:
                article = soup.getText()
                article = article[article.find('Home'):]
                article = article[:article.find('You can share this story on social media:')]
                article = article.replace('\xa0','').replace('\n','').replace('READ MORE','').replace('/','').replace('HomeWorld News','')
                art_list.append(article)

            day_list=[]
            month_list=[]
            year_list=[]
            year_control=[]

            for soup in list_soup:
                date_str = soup.find('span', {'class': 'date'}).text.strip()
                day, month, year, time = date_str.split()
                day = day.zfill(2)
                month = month.zfill(2).replace(',','')
                year= year

                day_list.append(day)
                month_list.append(month)
                year_list.append(year)
                

        dict_={'title':list_title,
                'article': art_list,
                'link': lst_link,            
                'year' : year_list,
                'month': month_list,
                'day': day_list
                }
        df= pd.DataFrame(dict_)
        return df
    
    except:
        pass

In [5]:
df_united_states = key_word('united states')
df_united_states

,title,article,link,year,month,day
0,North Korea warns of ‘overwhelming’ response —...,North Korea warns of ‘overwhelming’ re...,https://www.rt.com/news/572558-north-korea-ove...,2023,Mar,07
1,January 6 committee should be ‘tried for treas...,January 6 committee should be ‘tried f...,https://www.rt.com/news/572647-trump-january6-...,2023,Mar,08
2,German army struggling with NATO obligations –...,German army struggling with NATO oblig...,https://www.rt.com/news/572617-germany-lack-ta...,2023,Mar,07
3,US House Speaker to meet Taiwan’s leader – med...,US House Speaker to meet Taiwan’s lead...,https://www.rt.com/news/572619-top-republican-...,2023,Mar,08
4,Polish president claims Nord Stream blasts wer...,Polish president claims Nord Stream bl...,https://www.rt.com/news/572660-poland-nord-str...,2023,Mar,08
5,US spies concerned about Russia escalation ris...,US spies concerned about Russia escala...,https://www.rt.com/news/572651-us-spies-russia...,2023,Mar,08
6,Claims of Ukrainian link to Nord Stream sabota...,Claims of Ukrainian link to Nord Strea...,https://www.rt.com/news/572646-german-defense-...,2023,Mar,08
7,EU member state profiteering from Ukraine conf...,EU member state profiteering from Ukra...,https://www.rt.com/news/572532-bulgaria-weapon...,2023,Mar,06
8,Nazis are 'losers' – Schwarzenegger — RT Games...,HomeGames & Culture Nazis are 'losers' ...,https://www.rt.com/news/572601-us-schwarzenegg...,2023,Mar,07
9,"‘Havana syndrome’ has been debunked, but will ...","‘Havana syndrome’ has been debunked, b...",https://www.rt.com/news/572580-havana-syndrome...,2023,Mar,07


In [6]:
df_united_states.to_csv('rt_us.csv')  